In [1]:
!pip install selenium
!pip install beautifulsoup4
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

     |████████████████████████████████| 911kB 26.8MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ 

In [2]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re

In [3]:
# product name - 1 data
def get_product_name(soup):
    product_name = soup.find_all('div',{'class':'flex flex-auto _3-GQHh'})
    product_name = product_name[0].find_all('span')[0].get_text(separator=" ").strip() if len(product_name) > 0 else None
    return product_name

# Product rating - 1 data
def get_product_rating(soup):
    product_rating = soup.find_all('div',{'class':'flex _3A3c6_'})
    if len(product_rating) > 0:
        product_rating = product_rating[0].find_all('div',{'class':'OitLRu _1mYa1t'})[0]
        product_rating = float(product_rating.get_text(separator=" ").strip())
    else:
        product_rating = None
    return product_rating

# Product's number of review - 1 data
def get_product_num_review(soup):
    num_review = soup.find_all('div',{'class':'flex _3A3c6_'})
    if len(num_review) > 0:
        num_review = num_review[1].find_all('div',{'class':'OitLRu'})[0]
        num_review = num_review.get_text(separator=" ").strip()
    else:
        num_review = None
    return num_review

# num sold - 1 data
def get_num_sold(soup):
    num_sold = soup.find_all('div',{'class':'flex _210dTF'})
    if len(num_sold) > 0:
        num_sold = num_sold[0].find_all('div',{'class':'aca9MM'})[0]
        num_sold = num_sold.get_text(separator=" ").strip()
    else:
        num_sold = None
    return num_sold

# product price - 1 data
def get_product_price(soup):
    price_text = soup.find_all('div',{'class':'_3e_UQT'})
    if len(price_text) > 0:
        price_text = price_text[0].get_text(separator=" ").strip().split('-')[-1]
        price_text = re.findall("[0-9]", price_text)
        price_text = ''.join(price_text)
        price_text = float(price_text)
    else:
        price_text = None
    return price_text

# Stock Number - 1 data
def get_product_num_stock(soup):
    num_stock_div = soup.find_all('div',{'class':'flex items-center'})
    if len(num_stock_div) > 0:
        num_stock_data = num_stock_div[-1].find_all('div')[-1].get_text(separator=" ").strip()
        num_stock_data = float(num_stock_data.split(' ')[-2])
    return num_stock_data

# Store Name (1 data)
def get_store_name(soup):
    store_name_div = soup.find_all('div',{'class':'_3uf2ae'})
    store_name_text = store_name_div[0].text.strip() if len(store_name_div) > 0 else None
    return store_name_text

def get_store_total_review_num(soup):
    store_total_review_div = soup.find_all('span',{'class':'zw2E3N _2fK6RA'})
    total_review_num_text = store_total_review_div[0].text.strip() if len(store_total_review_div) > 0 else None
    return total_review_num_text

def get_store_total_product_num(soup):
    store_total_product_div = soup.find_all('span',{'class':'zw2E3N _2fK6RA'})
    store_total_product_text = store_total_product_div[1].text.strip() if len(store_total_product_div) > 0 else None
    return store_total_product_text

def get_store_response_rate(soup):
    store_response_rate_div = soup.find_all('span',{'class':'zw2E3N _2fK6RA'})
    store_response_rate_text = store_response_rate_div[2].text.strip() if len(store_response_rate_div) > 0 else None
    return store_response_rate_text

def get_store_response_duration(soup):
    store_response_duration_div = soup.find_all('span',{'class':'zw2E3N'})
    store_response_duration_text = store_response_duration_div[3].text.strip() if len(store_response_duration_div) > 0 else None
    return store_response_duration_text

def get_product_spec(soup):
    product_spec_keys = []
    product_spect_values = []
    specs_dict = dict()
    product_spec_div = soup.find_all('div',{'class':'aPKXeO'})
    if len(product_spec_div) > 1:
        product_spec_div = product_spec_div[1:]
        for product_spec_text in product_spec_div:
            spec_key = product_spec_text.find_all('label',{'class':'SFJkS3'})[0].text.strip()
            spec_value =  product_spec_text.find_all('div')[0].text.strip()
            # print(f'{spec_key}: {spec_value}')
            specs_dict.update({str(spec_key):spec_value})
    else:
        specs_dict = None
    return specs_dict

def get_product_category(soup):
    product_category_list = []
    product_category_div = soup.find_all('a',{'class':'_3YDLCj _3LWINq'})
    if len(product_category_div) > 0:
        for product_category_text in product_category_div:
            product_category = product_category_text.text.strip()
            product_category_list.append(product_category)
        product_category_list = product_category_list[-2]
    else:
        product_category_list = None
    return product_category_list

In [4]:
def get_link_detail_content(item_link, driver):
    driver.get(item_link)
    time.sleep(2)  # Allow 2 seconds for the web page to open
    scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
    i = 1
    
    while True:
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(scroll_pause_time)
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * i > scroll_height:
            break 

    time.sleep(5)    
    soup = BeautifulSoup(driver.page_source, "html.parser")

    product_name = get_product_name(soup)
    product_rating = get_product_rating(soup)
    product_num_review = get_product_num_review(soup)
    product_num_sold = get_num_sold(soup)
    product_price = get_product_price(soup)
    product_num_stock = get_product_num_stock(soup)
    product_specs_dict = get_product_spec(soup)
    product_category = get_product_category(soup)
    store_name = get_store_name(soup)
    store_total_num_review = get_store_total_review_num(soup)
    store_total_num_product = get_store_total_product_num(soup)
    store_response_rate = get_store_response_rate(soup)
    store_response_duration = get_store_response_duration(soup)
    store_location = None
    if product_specs_dict != None:
        if 'Dikirim Dari' in product_specs_dict.keys():
            store_location = product_specs_dict['Dikirim Dari']
    
    item_link_data =   {
        'Product Name': product_name, 'Product Rating': product_rating, 'Product Num Review': product_num_review, 
        'Product Num Sold': product_num_sold, 'Price': product_price, 'Product Num Stock': product_num_stock, 
        'Product Category': product_category, 'Product Specs': product_specs_dict,
        'Store Name':store_name, 'Store Total Num Review': store_total_num_review,
        'Store Total Num Product' : store_total_num_product, 'Store Response Rate':store_response_rate, 
        'Store Response Duration': store_response_duration, 'Store Location': store_location
        }
                    
    return item_link_data

In [5]:
def shopee_content_scraper(data_df, driver, idx_start=0, idx_stop=100):

    shopee_final_data = {
        'Product Name': [], 'Product Rating': [], 'Product Num Review': [], 
        'Product Num Sold': [], 'Price': [], 'Product Num Stock': [], 
        'Product Category': [], 'Product Specs': [],
        'Store Name':[], 'Store Total Num Review': [],
        'Store Total Num Product' : [], 'Store Response Rate':[], 
        'Store Response Duration': [], 'Store Location': [], 'Produk Link': []
        }

    all_link_list = data_df['Product Link'][idx_start:idx_stop].tolist()
    total_data = len(all_link_list)

    for idx, each_product_link in enumerate(all_link_list):
        print(f'[INFO] Scraping {idx+1}/{total_data} data')
        data_each_link = get_link_detail_content(each_product_link, driver)
        shopee_final_data['Product Name'].append(data_each_link['Product Name'])
        shopee_final_data['Product Rating'].append(data_each_link['Product Rating'])
        shopee_final_data['Product Num Review'].append(data_each_link['Product Num Review'])
        shopee_final_data['Product Num Sold'].append(data_each_link['Product Num Sold'])
        shopee_final_data['Price'].append(data_each_link['Price'])
        shopee_final_data['Product Num Stock'].append(data_each_link['Product Num Stock'])
        shopee_final_data['Product Category'].append(data_each_link['Product Category'])
        shopee_final_data['Product Specs'].append(data_each_link['Product Specs'])
        shopee_final_data['Store Name'].append(data_each_link['Store Name'])
        shopee_final_data['Store Total Num Review'].append(data_each_link['Store Total Num Review'])
        shopee_final_data['Store Total Num Product'].append(data_each_link['Store Total Num Product'])
        shopee_final_data['Store Response Rate'].append(data_each_link['Store Response Rate'])
        shopee_final_data['Store Response Duration'].append(data_each_link['Store Response Duration'])
        shopee_final_data['Store Location'].append(data_each_link['Store Location'])
        shopee_final_data['Produk Link'].append(each_product_link)

    shopee_final_data_df = pd.DataFrame.from_dict(shopee_final_data)
    shopee_final_data_df.to_csv('shopee_final_data.csv', index=False)
    return shopee_final_data_df

In [6]:
shopee_data = pd.read_csv('shopee_link_data.csv')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
shopee_final_data_res = shopee_content_scraper(shopee_data, driver, 20, 25)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


[INFO] Scraping 1/5 data
[INFO] Scraping 2/5 data
[INFO] Scraping 3/5 data
[INFO] Scraping 4/5 data
[INFO] Scraping 5/5 data


In [7]:
shopee_final_data_res

,Product Name,Product Rating,Product Num Review,Product Num Sold,Price,Product Num Stock,Product Category,Product Specs,Store Name,Store Total Num Review,Store Total Num Product,Store Response Rate,Store Response Duration,Store Location,Produk Link
0,VIVO Y12S 2021 RAM 3GB ROM 32GB Garansi Resmi ...,4.9,"1,4RB","2,2RB",1620000.0,1149.0,Handphone & Tablet,"{'Merek': 'Vivo', 'Model HP': 'VIVO Y12S', 'Ka...",3xj3ptefgf,"3,1RB",92,57%,hitungan jam,"KOTA JAKARTA BARAT - CENGKARENG, DKI JAKARTA, ID",https://shopee.co.id/VIVO-Y12S-2021-RAM-3GB-RO...
1,Infinix Hot 10 4/64GB + 4/128GB + 6/128GB Gara...,4.9,"2,6RB","3,8RB",1725000.0,121.0,Handphone & Tablet,"{'Merek': 'Infinix', 'Kamera Depan': '8 Mp', '...",eddy_metro,"51,4RB",155,100%,hitungan menit,"KOTA JAKARTA BARAT - TAMAN SARI, DKI JAKARTA, ID",https://shopee.co.id/Infinix-Hot-10-4-64GB-4-1...
2,"HP realme C12 3/32GB [6,000mAh Baterai, Triple...",4.9,"5,6RB","8,5RB",1499000.0,1189.0,Handphone & Tablet,"{'Merek': 'Realme', 'Kamera Depan': '5MP', 'Ka...",realme.official,137RB,39,57%,hitungan jam,"KOTA TANGERANG - CIPONDOH, BANTEN, ID",https://shopee.co.id/HP-realme-C12-3-32GB-6-00...
3,"Poco X3 NFC (8GB+128GB) Snapdragon™ 732G, Laya...",4.9,"22,7RB",10RB+,3299000.0,616.0,Handphone & Tablet,"{'Merek': 'Xiaomi', 'Kamera Depan': '20 MP', '...",xiaomi.official.id,"548,6RB",190,100%,hitungan menit,KOTA JAKARTA UTARA,https://shopee.co.id/Poco-X3-NFC-(8GB-128GB)-S...
4,Vivo Y12s hp RAM 3G ROM 32GB promo asli 100%or...,4.9,904,"1,4RB",1829000.0,1120.0,Handphone & Tablet,"{'Merek': 'Vivo', 'Model HP': 'Vivo Y12s', 'Ka...",threetoone,"18,7RB",99,100%,hitungan menit,"KOTA JAKARTA PUSAT - KEMAYORAN, DKI JAKARTA, ID",https://shopee.co.id/Vivo-Y12s-hp-RAM-3G-ROM-3...
